# Qubit spectroscopy: Rabi measurement example
This notebook demonstrates how to perform a Rabi experiment on a simulated or real quantum device. This experiment tests the calibration of the `RX` pulse by rotating through a full $2\pi$ radians and evaluating the excited state visibility as a function of the angle of rotation, $\theta$. The QUIL program for one data point for qubit 0 at, for example $\theta=\pi/2$, is

    DECLARE ro BIT[1]
    RX(pi/2) 0
    MEASURE 0 ro[0]
   
We expect to see a characteristic "Rabi flop" by sweeping $\theta$ over $[0, 2\pi)$, thereby completing a full rotation around the Bloch sphere. It should look like $\dfrac{1-\cos(\theta)}{2}$

## setup

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
from pyquil.api import get_qc, QuantumComputer

from forest_benchmarking.qubit_spectroscopy import fit_to_sinusoidal_waveform, run_rabi, sinusoidal_waveform

## measurement, analysis, and plotting

This method takes a `pandas.DataFrame` object with all the Rabi data and plots the Rabi flop. It fits to a sinusoid and evaluates the period for each qubit (which should be $2\pi$). 

In [ ]:
def plot_rabi_oscillations(df: pd.DataFrame) -> None:
    """
    Plot the Rabi results.
    """
    for q in df['qubit'].unique():
        df2 = df[df['qubit'] == q].sort_values('angle')
        angles = df2['angle']
        excited_state_visibilities = df2['avg']

        # plot raw data
        plt.plot(angles, excited_state_visibilities, 'o-', label=f"QC{q} Rabi data")

        try:
            # fit to sinusoid
            fit_params, _ = fit_to_sinusoidal_waveform(angles, excited_state_visibilities)
        except RuntimeError:
            # TODO: make into a warning rather than just printing it out
            print(f"Could not fit to experimental data for QC{q}")
        else:
            # overlay fitted sinusoidal curve
            plt.plot(angles, sinusoidal_waveform(angles, *fit_params),
                     label=f"QC{q} fitted line")

    plt.xlabel("RX angle [rad]")
    plt.ylabel("Excited state visibility")
    plt.title("Rabi flop")

    plt.legend(loc='best')
    plt.tight_layout()
    plt.savefig('rabi.png')
    plt.show()

In [ ]:
def main(qc: QuantumComputer):
    qubits = qc.qubits()
    
    results = run_rabi(qc=qc,
                       qubits=qubits,
                       filename='rabi.json')
    plot_rabi_oscillations(results)

In [ ]:
main(get_qc('9q-square-qvm'))